In [100]:
from sqlalchemy import text
from typing import List
from sqlalchemy.orm import Session
from db import SessionLocal
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from sklearn.linear_model import LinearRegression


from datetime import timedelta
from typing import Dict, Any
import numpy as np
import pandas as pd
from sqlalchemy.orm import Session
from sqlalchemy.sql import text
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from sklearn.linear_model import LinearRegression


def sales_predict(event_id: str, db: Session, n_future: int = 6, w: float = 0) -> Dict[str, Any]:
    # Fetch sales data
    result = db.execute(text("""
        SELECT o.created_at, pt.quantity, pt.quantity * tt.price AS sales
        FROM orders o
        JOIN purchased_tickets pt ON o.id = pt.order_id
        JOIN ticket_types tt ON o.event_id = tt.event_id
        WHERE o.event_id = :event_id
        ORDER BY o.created_at
    """), {"event_id": event_id}).fetchall()

    df = pd.DataFrame(result, columns=["created_at", "quantity", "price"])

    if df.empty:
        return {"historical": [], "predicted": []}

    # Historical sales
    sales = np.array(df["price"]).astype(int)
    time = np.arange(len(sales)).reshape(-1, 1)

    # Linear Regression Forecast
    lr = LinearRegression().fit(time, sales)
    time_future = np.arange(len(sales), len(sales) + n_future).reshape(-1, 1)
    lr_forecast_future = lr.predict(time_future)

    # Simple Exponential Smoothing Forecast
    ses_model = SimpleExpSmoothing(sales).fit(smoothing_level=0.5, optimized=False)
    ses_forecast_future = ses_model.forecast(n_future)

    # Combine (Hybrid Forecast)
    final_forecast_future = (1 - w) * ses_forecast_future + w * lr_forecast_future

    # Build date ranges
    last_date = pd.to_datetime(df["created_at"].iloc[-1])
    future_dates = [last_date + timedelta(days=i + 1) for i in range(n_future)]

    # Format data for chart
    historical_data = [
        {"date": str(d.date()), "sales": int(s)} for d, s in zip(df["created_at"], sales)
    ]
    predicted_data = [
        {"date": str(d.date()), "forecast": float(f)} for d, f in zip(future_dates, final_forecast_future)
    ]

    return {"historical": historical_data, "predicted": predicted_data}


In [101]:
db = SessionLocal()
event_id = '32f51670-5855-4296-89a8-b2016a7020a3'

print(sales_predict(event_id=event_id,db=db))

{'historical': [{'date': '2025-09-17', 'sales': 100}, {'date': '2025-09-19', 'sales': 600}, {'date': '2025-09-25', 'sales': 200}, {'date': '2025-09-25', 'sales': 100}, {'date': '2025-09-25', 'sales': 100}, {'date': '2025-09-25', 'sales': 200}, {'date': '2025-09-25', 'sales': 200}, {'date': '2025-09-25', 'sales': 100}, {'date': '2025-09-25', 'sales': 400}, {'date': '2025-09-25', 'sales': 100}, {'date': '2025-09-26', 'sales': 100}, {'date': '2025-09-26', 'sales': 100}, {'date': '2025-09-26', 'sales': 100}, {'date': '2025-10-30', 'sales': 300}], 'predicted': [{'date': '2025-10-31', 'forecast': 205.35888671875}, {'date': '2025-11-01', 'forecast': 205.35888671875}, {'date': '2025-11-02', 'forecast': 205.35888671875}, {'date': '2025-11-03', 'forecast': 205.35888671875}, {'date': '2025-11-04', 'forecast': 205.35888671875}, {'date': '2025-11-05', 'forecast': 205.35888671875}]}


In [102]:
result = db.execute(text("""
    SELECT o.created_at, pt.quantity, pt.quantity * tt.price AS sales
    FROM orders o
    JOIN purchased_tickets pt ON o.id = pt.order_id
    JOIN ticket_types tt ON o.event_id = tt.event_id
    WHERE o.event_id = :event_id
    ORDER BY o.created_at
"""), {"event_id": event_id}).fetchall()
df=pd.DataFrame(result)
df['created_at']=df['created_at'].dt.date
df.groupby('created_at')['sales'].sum()



created_at
2025-09-17     100.00
2025-09-19     600.00
2025-09-25    1400.00
2025-09-26     300.00
2025-10-30     300.00
Name: sales, dtype: object

In [103]:
from sqlalchemy import text
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from datetime import timedelta

def hybrid_forecast_api(event_id, db, n_future=6, w=0.5):
    # 1️⃣ Fetch data
    result = db.execute(text("""
        SELECT o.created_at, pt.quantity, pt.quantity * tt.price AS sales
        FROM orders o
        JOIN purchased_tickets pt ON o.id = pt.order_id
        JOIN ticket_types tt ON o.event_id = tt.event_id
        WHERE o.event_id = :event_id
        ORDER BY o.created_at
    """), {"event_id": event_id}).fetchall()

    df = pd.DataFrame(result, columns=['created_at', 'quantity', 'sales'])
    df['created_at'] = pd.to_datetime(df['created_at']).dt.date

    # 2️⃣ Group by date
    daily_sales = df.groupby('created_at')['sales'].sum().reset_index()

    # 3️⃣ Prepare data
    sales = daily_sales['sales'].astype(float).values
    time = np.arange(len(sales)).reshape(-1, 1)

    # 4️⃣ Fit Linear Regression
    lr = LinearRegression().fit(time, sales)

    # 5️⃣ Fit Simple Exponential Smoothing
    ses_model = SimpleExpSmoothing(sales).fit(smoothing_level=0.5, optimized=False)

    # 6️⃣ Forecast next n_future days
    future_dates = [daily_sales['created_at'].max() + timedelta(days=i+1) for i in range(n_future)]
    lr_forecast = [lr.predict([[len(sales)+i]])[0] for i in range(n_future)]
    ses_forecast = ses_model.forecast(n_future)
    final_forecast = [(1-w)*ses + w*lr for ses, lr in zip(ses_forecast, lr_forecast)]

    # 7️⃣ Convert to JSON-friendly format
    historical = daily_sales.rename(columns={'created_at':'date', 'sales':'sales'}).to_dict(orient='records')
    forecast = [{"date": str(d), "forecast_sales": float(f)} for d, f in zip(future_dates, final_forecast)]

    return [{"historical": historical, "forecast": forecast}, daily_sales]


In [104]:
forecast = hybrid_forecast_api(event_id=event_id, db=db, w=0.6)
forecast[0]



{'historical': [{'date': datetime.date(2025, 9, 17),
   'sales': Decimal('100.00')},
  {'date': datetime.date(2025, 9, 19), 'sales': Decimal('600.00')},
  {'date': datetime.date(2025, 9, 25), 'sales': Decimal('1400.00')},
  {'date': datetime.date(2025, 9, 26), 'sales': Decimal('300.00')},
  {'date': datetime.date(2025, 10, 30), 'sales': Decimal('300.00')}],
 'forecast': [{'date': '2025-10-31', 'forecast_sales': 519.5},
  {'date': '2025-11-01', 'forecast_sales': 525.5},
  {'date': '2025-11-02', 'forecast_sales': 531.5000000000001},
  {'date': '2025-11-03', 'forecast_sales': 537.5000000000001},
  {'date': '2025-11-04', 'forecast_sales': 543.5000000000002},
  {'date': '2025-11-05', 'forecast_sales': 549.5000000000002}]}

In [105]:
forecast[1]

,created_at,sales
0,2025-09-17,100.00
1,2025-09-19,600.00
2,2025-09-25,1400.00
3,2025-09-26,300.00
4,2025-10-30,300.00


In [106]:
from sqlalchemy import text
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from datetime import timedelta

def hybrid_forecast_api(event_id, db, n_future=6, w=0.5):
    # 1️⃣ Fetch data
    result = db.execute(text("""
        SELECT o.created_at, pt.quantity, pt.quantity * tt.price AS sales
        FROM orders o
        JOIN purchased_tickets pt ON o.id = pt.order_id
        JOIN ticket_types tt ON o.event_id = tt.event_id
        WHERE o.event_id = :event_id
        ORDER BY o.created_at
    """), {"event_id": event_id}).fetchall()

    df = pd.DataFrame(result, columns=['created_at', 'quantity', 'sales'])
    df['created_at'] = pd.to_datetime(df['created_at']).dt.date

    # 2️⃣ Group by date
    daily_sales = df.groupby('created_at')['sales'].sum().reset_index()

    # 3️⃣ Prepare data
    sales = daily_sales['sales'].astype(float).values
    time = np.arange(len(sales)).reshape(-1, 1)

    # 4️⃣ Fit Linear Regression
    lr = LinearRegression().fit(time, sales)

    # 5️⃣ Fit Simple Exponential Smoothing
    ses_model = SimpleExpSmoothing(sales).fit(smoothing_level=0.5, optimized=False)

    # 6️⃣ Forecast next n_future days
    future_dates = [daily_sales['created_at'].max() + timedelta(days=i+1) for i in range(n_future)]
    lr_forecast = [lr.predict([[len(sales)+i]])[0] for i in range(n_future)]
    ses_forecast = ses_model.forecast(n_future)
    final_forecast = [(1-w)*ses + w*lr for ses, lr in zip(ses_forecast, lr_forecast)]

    # 7️⃣ Convert historical to JSON-friendly format with day of week
    historical = []
    for _, row in daily_sales.iterrows():
        date = row['created_at']
        historical.append({
            "date": str(date),
            "day_of_week": pd.to_datetime(date).strftime("%A"),
            "sales": float(row['sales'])
        })

    # 8️⃣ Convert forecast to JSON-friendly format with day of week
    forecast = []
    for date, value in zip(future_dates, final_forecast):
        forecast.append({
            "date": str(date),
            "day_of_week": pd.to_datetime(date).strftime("%A"),
            "forecast_sales": float(value)
        })

    return {"historical": historical, "forecast": forecast}
hybrid_forecast_api(event_id=event_id, db=db, w=0.6)

{'historical': [{'date': '2025-09-17',
   'day_of_week': 'Wednesday',
   'sales': 100.0},
  {'date': '2025-09-19', 'day_of_week': 'Friday', 'sales': 600.0},
  {'date': '2025-09-25', 'day_of_week': 'Thursday', 'sales': 1400.0},
  {'date': '2025-09-26', 'day_of_week': 'Friday', 'sales': 300.0},
  {'date': '2025-10-30', 'day_of_week': 'Thursday', 'sales': 300.0}],
 'forecast': [{'date': '2025-10-31',
   'day_of_week': 'Friday',
   'forecast_sales': 519.5},
  {'date': '2025-11-01', 'day_of_week': 'Saturday', 'forecast_sales': 525.5},
  {'date': '2025-11-02',
   'day_of_week': 'Sunday',
   'forecast_sales': 531.5000000000001},
  {'date': '2025-11-03',
   'day_of_week': 'Monday',
   'forecast_sales': 537.5000000000001},
  {'date': '2025-11-04',
   'day_of_week': 'Tuesday',
   'forecast_sales': 543.5000000000002},
  {'date': '2025-11-05',
   'day_of_week': 'Wednesday',
   'forecast_sales': 549.5000000000002}]}

In [122]:
from sqlalchemy import text
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from datetime import timedelta

def hybrid_forecast_api(event_id, db, n_future=6, w=0.5):
    """
    Parameters:
    - event_id : str : event identifier
    - db : SQLAlchemy Session
    - n_future : int : number of days to forecast into the future
    - w : float : weight for Linear Regression in hybrid (0-1)
    """
    # 1️⃣ Fetch data
    result = db.execute(text("""
        SELECT o.created_at, pt.quantity, pt.quantity * tt.price AS sales
        FROM orders o
        JOIN purchased_tickets pt ON o.id = pt.order_id
        JOIN ticket_types tt ON o.event_id = tt.event_id
        WHERE o.event_id = :event_id
        ORDER BY o.created_at
    """), {"event_id": event_id}).fetchall()

    df = pd.DataFrame(result, columns=['created_at', 'quantity', 'sales'])
    df['created_at'] = pd.to_datetime(df['created_at']).dt.date

    # 2️⃣ Group by date
    daily_sales = df.groupby('created_at')['sales'].sum().reset_index()

    # 3️⃣ Prepare data
    sales = daily_sales['sales'].astype(float).values
    time = np.arange(len(sales)).reshape(-1, 1)

    # 4️⃣ Fit Linear Regression
    lr = LinearRegression().fit(time, sales)

    # 5️⃣ Fit Simple Exponential Smoothing
    ses_model = SimpleExpSmoothing(sales).fit(smoothing_level=0.5, optimized=False)

    # 6️⃣ Forecast next n_future days
    future_dates = [daily_sales['created_at'].max() + timedelta(days=i+1) for i in range(n_future)]
    lr_forecast = [lr.predict([[len(sales)+i]])[0] for i in range(n_future)]
    ses_forecast = ses_model.forecast(n_future)
    final_forecast = [(1-w)*ses + w*lr for ses, lr in zip(ses_forecast, lr_forecast)]

    # 7️⃣ Convert historical to JSON-friendly format with day of week
    historical = []
    for _, row in daily_sales.iterrows():
        date = row['created_at']
        historical.append({
            "date": str(date),
            "day_of_week": pd.to_datetime(date).strftime("%A"),
            "sales": float(row['sales'])
        })

    # 8️⃣ Convert forecast to JSON-friendly format with day of week
    forecast = []
    for date, value in zip(future_dates, final_forecast):
        forecast.append({
            "date": str(date),
            "day_of_week": pd.to_datetime(date).strftime("%A"),
            "forecast_sales": float(value)
        })

    return {"historical": historical, "forecast": forecast, "forecast_horizon_days": n_future}

len(hybrid_forecast_api(event_id=event_id, db=db, n_future=10000, w=0.6)['forecast'])

10000

In [121]:
import random
n=10000000
r=10
c=0
c1=0
for _ in range(n):
    x,y=random.randrange(0,r), random.randrange(0,r)
    if x**2+y**2<r:
        c+=1
    else:
        c1+=1

r



10

In [78]:
x,y

(2, 5)